In [12]:
%store -r remaining_prot
%store -r remaining_fat
%store -r remaining_carb

In [13]:
# imports
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
# load data for training


df= pd.read_csv('preprocessed.csv')
df

In [18]:
features = df.drop(columns=['Unnamed: 0','steps','n_ingredients']) 
features.head(3)
type(features.fats)
features.info()
features

In [16]:
# defining thresholds
remaining_prot= int(individual_ideal['daily_protein_cals']-individual_junk['protein'])
remaining_fat= int(individual_ideal['daily_fat_cals']-individual_junk['carb'])
remaining_carb= int(individual_ideal['daily_carb_cals']-individual_junk['fats'])

features.loc[features['fats']>remaining_fat,'fat_category']='High fat'
features.loc[features['fats']<=remaining_fat,'fat_category']='Low fat'

features.loc[features['carbs']>remaining_prot,'carbs_category']='High carbs'
features.loc[features['carbs']<=remaining_prot,'carbs_category']='Low carbs'

features.loc[features['protein']>remaining_fat,'protein_category']='High protein'
features.loc[features['protein']<=remaining_fat,'protein_category']='Low protein'

In [17]:
X=features
X

In [ ]:
# Convert the numeric values of macros into labels
def label_macros(row):
    if row['carbs'] > 0.5:
        row['carbs_label'] = 'high carb'
    else:
        row['carbs_label'] = 'low carb'
    if row['protein'] > 0.5:
        row['protein_label'] = 'high protein'
    else:
        row['protein_label'] = 'low protein'
    if row['fats'] > 0.5:
        row['fat_label'] = 'high fat'
    else:
        row['fat_label'] = 'low fat'
  
    return row

X =X.apply(label_macros, axis=1)


In [ ]:
X=features
X

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Define a function to calculate macro repartition scores
def macro_score(row):
    if row['carbs_category'] == 'high carb':
        carb_score = 1
    else:
        carb_score = 0
    if row['protein_category'] == 'high protein':
        protein_score = 1
    else:
        protein_score = 0
    if row['fat_category'] == 'high fat':
        fat_score = 1
    else:
        fat_score = 0
   
    macro_score = carb_score + protein_score + fat_score
    return macro_score

# Calculate macro scores for each recipe
X['macro_score'] = X.apply(macro_score, axis=1)

# Define a function to calculate favorite food similarity scores
def favorite_food_scores(recipe1, recipe2):
    # Use TF-IDF to vectorize recipe ingredients
    vectorizer = TfidfVectorizer()
    ingredients = [recipe1['ingredients'], recipe2['ingredients']]
    vectors = vectorizer.fit_transform(ingredients)
    
    # Calculate cosine similarity between ingredient vectors
    similarity = cosine_similarity(vectors)
    
    return similarity[0][1]

# Define a function to calculate total scores for each recipe
def calculate_scores(recipe1, recipe2):
    macro_score = recipe1['macro_score'] + recipe2['macro_score']
    favorite_food_score = favorite_food_scores(recipe1, recipe2)
    total_score = macro_score + favorite_food_score
    return total_score

# Example usage: compare two recipes by calculating their scores
recipe1 = X.loc[0]
recipe2 = X.loc[1]
score = calculate_scores(recipe1, recipe2)
print(score)


In [ ]:
X